In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.ulaval.ca/etudes/programmes/baccalaureat-en-genie-mecanique#section-structure"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [7]:
page_soup = soup(driver.page_source, 'lxml')

In [10]:
#Genie Mecanique Regle 1
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[0].findAll("div", {"class": "toggle-section"})[1].find("ul", {"class": "fe--liste-cours"}).findAll("a")
len(containers)

2

In [11]:
#Cheminement sans concentration Regle 1
containers += page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[1].findAll("div", {"class": "toggle-section"})[0].findAll("a")
len(containers)

57

In [12]:
#Cheminement sans concentration Regle 2
containers += page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[1].findAll("div", {"class": "toggle-section"})[1].findAll("a")
len(containers)

59

In [13]:
#Cheminement sans concentration Regle 3
containers += page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[1].findAll("div", {"class": "toggle-section"})[2].findAll("a")
len(containers)

60

In [14]:
link_texts = [container.text.strip() for container in containers]
link_texts

['GMN-2902\nSanté et sécurité pour ingénieur III',
 'MED-1100\nSanté et sécurité au travail : notions de base',
 "GMC-4150\nIntroduction à l'aérodynamique",
 'GMC-4151\nPropulsion aéronautique et spatiale',
 "PHY-2100\nSciences de l'espace",
 'ARC-2001\nArchitecture et environnement',
 'ARC-3103\nAcoustique architecturale',
 'GBO-2040\nCharpentes en bois I',
 'GBO-4070\nEnveloppe du bâtiment',
 'GCI-2004\nStructures de béton',
 'GCI-2007\nStructures métalliques',
 'GCI-3001\nImpacts environnementaux',
 'GCI-3002\nGestion des eaux',
 'GEL-2003\nÉlectrotechnique',
 'GEL-4100\nCommande industrielle',
 "GEL-4799\nDangers de l'électricité",
 'GMC-3012\nIntroduction à la mécanique du bâtiment et aux énergies renouvelables',
 'GMC-3015\nConception intégrée des systèmes CVAC-R',
 'GML-4151\nIntroduction aux matériaux pour la santé',
 "GIN-2010\nGestion opérationnelle des systèmes d'entreprise",
 'GIN-2110\nOptimisation des systèmes et des réseaux',
 'GIN-3010\nConception et implantation des sy

# 2. Implement automation script to scrape all courses

first click open all tabs

In [16]:
driver.find_element_by_id("genie-mecanique-regle-1").click()

In [20]:
from selenium.webdriver.common.keys import Keys
tab = driver.find_element_by_id("cheminement-sans-concentration-regle-1")
time.sleep(2)
tab.send_keys(Keys.DOWN)
tab.click()

In [22]:
tab = driver.find_element_by_id("cheminement-sans-concentration-regle-2")
time.sleep(2)
tab.send_keys(Keys.DOWN)
tab.click()

In [25]:
tab = driver.find_element_by_id("cheminement-sans-concentration-regle-3")
time.sleep(3)
tab.send_keys(Keys.DOWN)
tab.click()

In [26]:
#scroll to the top of the page
tab.send_keys(Keys.CONTROL + Keys.HOME)

In [27]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title_container = page_soup.find("h1", {"class": "fe--titre"})
    course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
    course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
    course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  GMN-2902 Santé et sécurité pour ingénieur III 
 Problèmes de santé et de sécurité du travail, lois et règlements. Accidents et maladies professionnelles. Éléments d'ergonomie et postures de travail. Conséquences des accidents, incidents et maladies professionnelles. Organisation de la sécurité en entreprise. Systèmes de prévention. Évaluation en sécurité. SIMDUT. Notions du risque. Types d'expositions. Fondements de normes. Évaluation des risques associés aux poussières, aux bruits, aux vibrations, à la contrainte thermique et à l'éclairage. Méthodes de mesure, contrôle et protection, normes et règlements. Sécurité industrielle. Mesure du niveau de danger. Acceptabilité du risque. Gestion du risque. Santé publique, sécurité et environnement. Devoirs et responsabilités juridiques des ingénieurs. Lois, normes et règlements en matière de sécurité, risques et gestion de risques. Maladies infectieuses et santé des travailleurs expatriés.
1 courses scraped

Scraped  MED-1100 Santé e

Scraped  GEL-4799 Dangers de l'électricité 
 Cette formation est requise pour l'accès à quelques-uns de nos laboratoires (locaux), et donc pour certains cours. Elle est exigée pour des raisons de norme technique de sécurité.
16 courses scraped

Scraped  GMC-3012 Introduction à la mécanique du bâtiment et aux énergies renouvelables 
 Ce cours constitue une introduction à la mécanique du bâtiment en abordant les thèmes suivants : conditionnement de l’air, air humide, confort thermique, qualité de l’air intérieur, thermique du bâtiment, charge de chauffage, rayonnement solaire, charge de refroidissement, simulation énergétique, réseaux de distribution de fluide, éclairage. De plus, une partie du cours est consacrée aux énergies renouvelables, en particulier à la géothermie et à l’énergie solaire. Finalement, le cours propose à l'étudiant de réfléchir sur l’importance de l’énergie dans nos vies, dans un contexte de développement durable.
17 courses scraped

Scraped  GMC-3015 Conception int

Scraped  GMC-4201 Ingénierie de la qualité 
 Le cours se divise en trois parties. Première partie : contrôle de la qualité de la conformité : métrologie; contrôle dimensionnel des longueurs et des angles, contrôle géométrique de rectitude, de planéité, de circularité, de parallélisme, de perpendicularité, de localisation et autres; inspection manuelle, automatisée et par calibres à limites. Deuxième partie : contrôle statistique : notions de base, échantillonnage, contrôle par lots et en cours de procédé, analyse de capacité; implantation et outils d'application des techniques statistiques. Troisième partie : assurance de la qualité : normes ISO-9000, documentation du système qualité; implantation et amélioration de la qualité. Plusieurs normes sont présentées. Quatre laboratoires complètent la formation pratique.
29 courses scraped

Scraped  GMC-4202 Procédés et développement de produits 
 Approfondissement de la fabrication mécanique. Outils pour le développement de produits et de pr

Scraped  GEL-4250 Commande des systèmes avancée 
 Représentation d'état : représentation continue et discrète, propriétés, observabilité et gouvernabilité. Observateurs : Luenberger, filtre de Kalman. Commande modale : continue et discrète, ajout d'une action intégrale, ajout d'un observateur. Commande linéaire quadratique (LQ) : continue et discrète, ajout d'une action intégrale, ajout d'un filtre de Kalman (commande LQ Gaussienne). Commande prédictive : principe de l'horizon fuyant, critère de commande, équations de prédiction, paramètres de réglage, contraintes. Commande robuste : performances fréquentielles, incertitudes, stabilité et performance robustes, design des régulateurs.
41 courses scraped

Scraped  GEL-4799 Dangers de l'électricité 
 Cette formation est requise pour l'accès à quelques-uns de nos laboratoires (locaux), et donc pour certains cours. Elle est exigée pour des raisons de norme technique de sécurité.
42 courses scraped

Scraped  GIF-1003 Programmation avancée en

Scraped  ENT-2023 Entrepreneuriat technologique 
 Ce cours vise à acquérir une compétence distinctive de gestion d'entreprises technologiques. Il permet à l'étudiant de se familiariser avec les particularités du processus de création et de développement de telles entreprises.
55 courses scraped

Scraped  ENT-3000 Portfolio entrepreneurial I 
 Préalables : être inscrit au profil entrepreneurial ou au microprogramme en développement des compétences entrepreneuriales et détenir une moyenne égale ou supérieure à 2,67 calculée sur 24 crédits et plus. Le portfolio entrepreneurial permet à l'étudiant de développer un projet lié à son domaine d'études, d'établir un réseau de contacts et d'améliorer ses compétences entrepreneuriales. Un soutien personnalisé est assuré par un conseiller d'Entrepreneuriat Laval tout au long du développement du projet. L'accompagnement facilite la démarche de l'étudiant et lui permet de valider et d'approfondir son idée de projet, d'orienter ses démarches et d'éla

# 3. Inspect and write to CSV

In [28]:
course_codes

['GMN-2902',
 'MED-1100',
 'GMC-4150',
 'GMC-4151',
 'PHY-2100',
 'ARC-2001',
 'ARC-3103',
 'GBO-2040',
 'GBO-4070',
 'GCI-2004',
 'GCI-2007',
 'GCI-3001',
 'GCI-3002',
 'GEL-2003',
 'GEL-4100',
 'GEL-4799',
 'GMC-3012',
 'GMC-3015',
 'GML-4151',
 'GIN-2010',
 'GIN-2110',
 'GIN-3010',
 'GIN-3110',
 'GIN-3220',
 'GSO-2105',
 'GSO-3103',
 'GCH-1007',
 'GMC-4200',
 'GMC-4201',
 'GMC-4202',
 'GIF-4101',
 'GLO-2100',
 'IFT-4102',
 'GMC-4250',
 'GML-2007',
 'GML-2250',
 'GML-2251',
 'GML-3000',
 'GEL-1000',
 'GEL-1002',
 'GEL-4250',
 'GEL-4799',
 'GIF-1003',
 'GLO-4001',
 'GMC-3300',
 'GMC-3351',
 'IFT-1700',
 'GAE-2005',
 'GMC-4048',
 'GMC-4054',
 'GMC-4100',
 'GMC-4350',
 'GMC-4351',
 'ENT-1000',
 'ENT-2023',
 'ENT-3000',
 'ENT-3010',
 'GMC-3550',
 'GMC-3551',
 'ANL-2020']

In [29]:
course_names

['Santé et sécurité pour ingénieur III',
 'Santé et sécurité au travail : notions de base',
 "Introduction à l'aérodynamique",
 'Propulsion aéronautique et spatiale',
 "Sciences de l'espace",
 'Architecture et environnement',
 'Acoustique architecturale',
 'Charpentes en bois I',
 'Enveloppe du bâtiment',
 'Structures de béton',
 'Structures métalliques',
 'Impacts environnementaux',
 'Gestion des eaux',
 'Électrotechnique',
 'Commande industrielle',
 "Dangers de l'électricité",
 'Introduction à la mécanique du bâtiment et aux énergies renouvelables',
 'Conception intégrée des systèmes CVAC-R',
 'Introduction aux matériaux pour la santé',
 "Gestion opérationnelle des systèmes d'entreprise",
 'Optimisation des systèmes et des réseaux',
 'Conception et implantation des systèmes de production',
 'Environnement et cycle de vie des produits',
 'Analyse des risques, fiabilité et maintenance',
 'Achat et approvisionnement',
 'Ingénierie de la chaîne logistique',
 'Procédés de mise en forme de

In [30]:
course_descs

["Problèmes de santé et de sécurité du travail, lois et règlements. Accidents et maladies professionnelles. Éléments d'ergonomie et postures de travail. Conséquences des accidents, incidents et maladies professionnelles. Organisation de la sécurité en entreprise. Systèmes de prévention. Évaluation en sécurité. SIMDUT. Notions du risque. Types d'expositions. Fondements de normes. Évaluation des risques associés aux poussières, aux bruits, aux vibrations, à la contrainte thermique et à l'éclairage. Méthodes de mesure, contrôle et protection, normes et règlements. Sécurité industrielle. Mesure du niveau de danger. Acceptabilité du risque. Gestion du risque. Santé publique, sécurité et environnement. Devoirs et responsabilités juridiques des ingénieurs. Lois, normes et règlements en matière de sécurité, risques et gestion de risques. Maladies infectieuses et santé des travailleurs expatriés.",
 "L'objectif de ce cours est d'initier l'étudiant aux notions de base des différentes disciplines

In [31]:
len(list(dict.fromkeys(course_codes)))

59

In [32]:
course_codes1 = []
course_names1 = []
course_descs1 = []
counter = 0

for i in range(len(course_codes)):
    if course_codes[i] not in course_codes1:
        course_codes1.append(course_codes[i])
        course_names1.append(course_names[i])
        course_descs1.append(course_descs[i])
        counter += 1
    else:
        print("Duplicate: ", course_names[i])
        
print(counter)

Duplicate:  Dangers de l'électricité
59


In [33]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes1,
    "Course Name": course_names1,
    "Course Description": course_descs1
})

df

,Course Number,Course Name,Course Description
0,GMN-2902,Santé et sécurité pour ingénieur III,"Problèmes de santé et de sécurité du travail, ..."
1,MED-1100,Santé et sécurité au travail : notions de base,L'objectif de ce cours est d'initier l'étudian...
2,GMC-4150,Introduction à l'aérodynamique,La statique de l'atmosphère. Équations intégra...
3,GMC-4151,Propulsion aéronautique et spatiale,Application des principes fondamentaux de la m...
4,PHY-2100,Sciences de l'espace,Ce cours s'adresse aux étudiants inscrits à un...
5,ARC-2001,Architecture et environnement,Ce cours présente les notions fondamentales de...
6,ARC-3103,Acoustique architecturale,Ce cours vise la compréhension des principes a...
7,GBO-2040,Charpentes en bois I,"Notions préliminaires : formes structurales, p..."
8,GBO-4070,Enveloppe du bâtiment,Ce cours comprend les fondements de l'envelopp...
9,GCI-2004,Structures de béton,Théorie linéaire. Théorie à la rupture : règle...


In [34]:
df.to_csv('Laval_GenieMecanique_Elective_Courses.csv', index = False)

In [35]:
driver.quit()